# Day 6: Guard Gallivant

## Import libraries

In [ ]:
import copy

## Import data

In [ ]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - The whole puzzle input represents a lab (grid map) patrolled by a guard.
# - '^': Represents the guard facing upwards.
# - '#': Represents any obstructions (crates, desks, etc.). 
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-6_input.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

# Separate data by line to create rows for grid
grid = file_data.split("\n")

# Separate data in EACH row to represent EACH column
for i in range(len(grid)):
    grid[i] = list(grid[i])
    

print(grid)
# ====================================================================================================================

## Helper functions

In [ ]:
def next_pos_is_obstacle(_grid, _direction, _guardCurrentRow, _guardCurrentCol):
    if _direction == 'up':
        # Check if the next position *upwards* is an obstacle
        #print(_grid[_guardCurrentRow - 1][_guardCurrentCol])
        if _grid[_guardCurrentRow - 1][_guardCurrentCol] == '#': return True

    if _direction == 'down':
        if _grid[_guardCurrentRow + 1][_guardCurrentCol] == '#': return True

    if _direction == 'left':
        if _grid[_guardCurrentRow][_guardCurrentCol - 1] == '#': return True

    if _direction == 'right':
        if _grid[_guardCurrentRow][_guardCurrentCol + 1] == '#': return True

    return False

In [ ]:
'''This script reads the input from a file named input.txt, where each line represents a row in the grid. The get_guard_path function simulates the guard's movement and returns the set of visited positions. The get_loop_positions function checks each position in the visited set to see if adding a new obstruction would cause the guard to get stuck in a loop. If so, it adds the position to the loop_positions set. Finally, the script prints the number of positions in the loop_positions set.

Note that this script assumes that the input file is in the same directory as the script. You may need to modify the file path if your input file is located elsewhere.'''

# Need to test the function below - today !

from collections import defaultdict

def get_guard_path(grid):
    directions = [(0, -1), (1, 0), (0, 1), (-1, 0)]  # up, right, down, left
    guard_position = None
    guard_direction = 0  # up

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == '^':
                guard_position = (i, j)
                break
        if guard_position:
            break

    visited = set()
    visited.add(guard_position)

    while True:
        next_position = (guard_position[0] + directions[guard_direction][0], guard_position[1] + directions[guard_direction][1])

        if next_position[0] < 0 or next_position[0] >= len(grid) or next_position[1] < 0 or next_position[1] >= len(grid[0]) or grid[next_position[0]][next_position[1]] == '#':
            guard_direction = (guard_direction + 1) % 4
            next_position = (guard_position[0] + directions[guard_direction][0], guard_position[1] + directions[guard_direction][1])

        if next_position[0] < 0 or next_position[0] >= len(grid) or next_position[1] < 0 or next_position[1] >= len(grid[0]):
            break

        visited.add(next_position)
        guard_position = next_position

    return visited

def get_loop_positions(grid):
    visited = get_guard_path(grid)
    loop_positions = set()

    for position in visited:
        if position == (0, 0):
            continue

        for direction in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
            next_position = (position[0] + direction[0], position[1] + direction[1])

            if next_position in visited:
                continue

            new_grid = [row[:] for row in grid]
            new_grid[next_position[0]][next_position[1]] = '#'

            new_visited = get_guard_path(new_grid)

            if len(new_visited) < len(visited):
                loop_positions.add(next_position)

    return loop_positions

def main():
    with open('input.txt', 'r') as f:
        grid = [list(line.strip()) for line in f.readlines()]

    loop_positions = get_loop_positions(grid)
    print(len(loop_positions))

if __name__ == "__main__":
    main()

## Part 1

In [ ]:
# *** [PART 1] ***
# ! PROBLEM: Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:
# - If there is something (e.g. an obstacle = '#') directly in front of a guard, turn right 90 degrees.
# - Otherwise, take a step forward.
# - TODO: Determine which specific positions in the lab (grid map) will be in the patrol path by predicting the guard's route and calculate the total number of distinct steps taken by the guard in the patrol path.
# - NOTE: The positions (including the guard's starting position) visited by the guard before leaving the area are marked with an 'X':
# - NOTE: A guard leaves the lab (grid map) after reaching the end of any of the 4 sides of the grid map. 
# ====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part1_grid = copy.deepcopy(grid)

nRows = len(part1_grid)
nCols = len(part1_grid[0])
guardCurrentRow = 0; guardCurrentCol = 0
direction = '' # string var to store the name of the current direction of the guard
steps = 0 # int var to store the total number of steps the guard takes in her patrol path.

# Determine initial position of guard
for rowIdx in range(nRows):
    for colIdx in range(nCols):
        if part1_grid[rowIdx][colIdx] == '^':
            # Set guard's initial position
            guardCurrentRow = rowIdx
            guardCurrentCol = colIdx
            direction = 'up'

# While the guard has NOT reached the END of any 1 of the 4 sides of the lab (grid)
while 0 < guardCurrentRow < (nRows - 1) and 0 < guardCurrentCol < (nCols - 1):
    # Mark current position as visited
    part1_grid[guardCurrentRow][guardCurrentCol] = 'X'
    # Check if the NEXT position (in relation to the CURRENT position) is an obstacle
    hitObstacle = next_pos_is_obstacle(part1_grid, direction, guardCurrentRow, guardCurrentCol)

    if direction == 'up':
        if hitObstacle == False:
            guardCurrentRow -= 1 # Move guard 1 position [up]
            #grid[guardCurrentRow][guardCurrentCol] = 'X'
        elif hitObstacle == True:
            # Turn guard right 90 degrees
            direction = 'right'
    # NOTE: 'elif' used to prevent previous if statement to continue to this (and remaining) if statements if not applicable
    elif direction == 'down':
        # Check if next position is an obstacle
        if hitObstacle == False:
            guardCurrentRow += 1 # Move guard 1 position [down]
        elif hitObstacle == True:
            # Turn guard right 90 degrees
            direction = 'left'

    elif direction == 'left':
        # Check if next position is an obstacle
        if hitObstacle == False:
            guardCurrentCol -= 1 # Move guard 1 position [left]
        elif hitObstacle == True:
            # Turn guard right 90 degrees
            direction = 'up'

    elif direction == 'right':
        # Check if next position is an obstacle
        if hitObstacle == False:
            guardCurrentCol += 1 # Move guard 1 position [right]
        elif hitObstacle == True:
            # Turn guard right 90 degrees
            direction = 'down'

    # while loop BREAK condition
    if guardCurrentRow == 0 or guardCurrentRow == (nRows - 1) or guardCurrentCol == 0 or guardCurrentCol == (nCols - 1):
        part1_grid[guardCurrentRow][guardCurrentCol] = 'X' # Mark last visited position
        break

# Count the total number of DISTINCT visited areas in the grid
for row in part1_grid:
    for col in row:
        if col == "X":
            steps += 1

print("Total number of distinct areas visited (PART 1):", steps)
# ====================================================================================================================

## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search. To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.
# - NOTE: In the given example (see website), there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop.
# - TODO: It doesn't really matter what you choose to use as an obstacle (0, *, %, etc.), as long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes. You need to get the guard stuck in a LOOP by adding a SINGLE new obstruction. How many different positions could you choose for this obstruction?
#====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part2_grid = copy.deepcopy(grid)